In [1]:
import pandas as pd
import numpy as np
import os
import glob
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

DATA_DIR = '../data'  
MODEL_DIR = '../models'
MIN_DATA_POINTS = 252 

SP500_TICKER_FILE = os.path.join(DATA_DIR, 'sp500_tickers_correct.csv')

os.makedirs(MODEL_DIR, exist_ok=True)

def get_sp500_tickers_from_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        if 'Symbol' not in df.columns:
            print(f"HATA: '{file_path}' dosyasında 'Symbol' sütunu bulunamadı.")
            return None
        
        tickers = df['Symbol'].tolist()
        print(f"'{file_path}' dosyasından {len(tickers)} adet S&P 500 hisse senedi sembolü başarıyla okundu.")
        return tickers
    except FileNotFoundError:
        print(f"HATA: S&P 500 ticker dosyası bulunamadı: '{file_path}'")
        print("Lütfen önce 'scrape_sp500.py' dosyasını çalıştırdığınızdan emin olun.")
        return None
    except Exception as e:
        print(f"HATA: S&P 500 listesi okunurken bir hata oluştu: {e}")
        return None

In [2]:
def load_and_clean_data(file_path):
    try:
        df = pd.read_csv(file_path)
        stock_ticker = os.path.basename(file_path).split('.')[0]
        print(f"--- {stock_ticker} verisi işleniyor... ---")

        required_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
        if not all(col in df.columns for col in required_cols):
            print(f"HATA: {stock_ticker}.csv dosyasında gerekli sütunlar bulunamadı. Atlanıyor.")
            return None, None

        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        
        df.dropna(how='any', inplace=True)

        price_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
        df = df[(df[price_cols] > 0).all(axis=1)]

        if len(df) < MIN_DATA_POINTS:
            print(f"UYARI: {stock_ticker} için yeterli veri yok ({len(df)} satır). Atlanıyor.")
            return None, None
        
        return df, stock_ticker
    except Exception as e:
        print(f"HATA: {file_path} işlenirken bir hata oluştu: {e}. Atlanıyor.")
        return None, None

In [3]:
def feature_engineering(df):
    df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()
    df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()

    delta = df['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift())
    low_close = np.abs(df['Low'] - df['Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)
    df['ATR'] = true_range.rolling(window=14).mean()

    df['Target'] = (df['Close'] > df['Open']).astype(int)
    df['Target'] = df['Target'].shift(-1)
    df.dropna(inplace=True)
    df['Target'] = df['Target'].astype(int)
    
    return df

In [4]:
def create_windowed_dataset(X, y, window_size=5):
    X_windowed, y_windowed = [], []
    for i in range(window_size, len(X)):
        features = X.iloc[i-window_size:i].values.flatten()
        X_windowed.append(features)
        y_windowed.append(y.iloc[i])
    return np.array(X_windowed), np.array(y_windowed)

from xgboost import XGBClassifier

def run_experiments_for_stock(df, stock_ticker):
    X_full = df.drop('Target', axis=1)
    y_full = df['Target']
    stock_results = []
    
    feature_subsets = { 'Tum_Ozellikler': X_full.columns.tolist(), 'Sadece_Teknik_Indikatorler': ['EMA_10', 'EMA_20', 'EMA_50', 'RSI', 'ATR'] }
    k_values = [5, 10, 15]
    window_sizes = [3, 7]
    split_ratio = 0.8
    
    for subset_name, subset_columns in feature_subsets.items():
        X_subset = X_full[subset_columns]
        for k in k_values:
            current_k = min(k, X_subset.shape[1])
            if k > current_k: continue
            for window in window_sizes:
                print(f"\n--- Deney: {subset_name} | k={current_k} | Pencere={window} ---")
                
                selector = SelectKBest(f_classif, k=current_k).fit(X_subset, y_full)
                X_selected = X_subset[X_subset.columns[selector.get_support()]]
                X_windowed, y_windowed = create_windowed_dataset(X_selected, y_full, window_size=window)
                if len(X_windowed) == 0: continue
                
                split_index = int(len(X_windowed) * split_ratio)
                X_train_w, X_test_w = X_windowed[:split_index], X_windowed[split_index:]
                y_train_w, y_test_w = y_windowed[:split_index], y_windowed[split_index:]
                
                scaler = StandardScaler().fit(X_train_w)
                X_train_scaled = scaler.transform(X_train_w)
                X_test_scaled = scaler.transform(X_test_w)
                
                model = XGBClassifier(eval_metric='logloss', random_state=42).fit(X_train_scaled, y_train_w)
                y_pred = model.predict(X_test_scaled)
                accuracy = accuracy_score(y_test_w, y_pred)
                
                model_filename = os.path.join(MODEL_DIR, f"xgb_{stock_ticker}_subset-{subset_name}_k{current_k}_w{window}.joblib")
                joblib.dump(model, model_filename)
                stock_results.append({ 'model_tipi': 'XGBoost', 'hisse_senedi': stock_ticker, 'alt_kume': subset_name, 'k_ozellik_sayisi': current_k, 'pencere_boyutu': window, 'dogruluk': accuracy, 'model_dosyasi': model_filename })
    
    print(f"==> {stock_ticker} için {len(stock_results)} deneyi tamamlandı.")
    return stock_results

In [5]:
def main():
    sp500_tickers = get_sp500_tickers_from_csv(SP500_TICKER_FILE)
    if sp500_tickers is None:
        return 
    
    all_csv_files = glob.glob(os.path.join(DATA_DIR, '*.csv'))
    if not all_csv_files:
        print(f"UYARI: '{DATA_DIR}' klasöründe hiçbir .csv dosyası bulunamadı.")
        return

    target_csv_files = []
    sp500_set = set(sp500_tickers)
    for file_path in all_csv_files:
        ticker = os.path.basename(file_path).split('.')[0]
        if ticker in sp500_set:
            target_csv_files.append(file_path)
            
    print(f"\nİşlenecek {len(target_csv_files)} adet S&P 500 hissesi bulundu.")

    all_results = []
    for file_path in target_csv_files:
        df_clean, stock_ticker = load_and_clean_data(file_path)
        if df_clean is None:
            continue
            
        df_featured = feature_engineering(df_clean)
        if len(df_featured) < MIN_DATA_POINTS:
            print(f"UYARI: {stock_ticker} için özellik mühendisliği sonrası yeterli veri kalmadı. Atlanıyor.")
            continue
        
        results_for_one_stock = run_experiments_for_stock(df_featured, stock_ticker)
        all_results.extend(results_for_one_stock)

    print("\n--- TÜM HİSSE SENETLERİ İÇİN TÜM DENEYLER TAMAMLANDI ---")

    if not all_results:
        print("Hiçbir hisse senedi için başarılı bir deney sonucu elde edilemedi.")
        return
        
    results_df = pd.DataFrame(all_results)
    results_df_sorted = results_df.sort_values(by=['hisse_senedi', 'dogruluk'], ascending=[True, False])

    print("\nBirleştirilmiş Deney Sonuçları Tablosu:")
    display(results_df_sorted)

if __name__ == '__main__':
    main()

'../data/sp500_tickers_correct.csv' dosyasından 503 adet S&P 500 hisse senedi sembolü başarıyla okundu.

İşlenecek 472 adet S&P 500 hissesi bulundu.
--- MGM verisi işleniyor... ---

--- Deney: Tum_Ozellikler | k=5 | Pencere=3 ---

--- Deney: Tum_Ozellikler | k=5 | Pencere=7 ---

--- Deney: Tum_Ozellikler | k=10 | Pencere=3 ---

--- Deney: Tum_Ozellikler | k=10 | Pencere=7 ---

--- Deney: Sadece_Teknik_Indikatorler | k=5 | Pencere=3 ---

--- Deney: Sadece_Teknik_Indikatorler | k=5 | Pencere=7 ---
==> MGM için 6 deneyi tamamlandı.
--- FAST verisi işleniyor... ---

--- Deney: Tum_Ozellikler | k=5 | Pencere=3 ---

--- Deney: Tum_Ozellikler | k=5 | Pencere=7 ---

--- Deney: Tum_Ozellikler | k=10 | Pencere=3 ---

--- Deney: Tum_Ozellikler | k=10 | Pencere=7 ---

--- Deney: Sadece_Teknik_Indikatorler | k=5 | Pencere=3 ---

--- Deney: Sadece_Teknik_Indikatorler | k=5 | Pencere=7 ---
==> FAST için 6 deneyi tamamlandı.
--- WFC verisi işleniyor... ---

--- Deney: Tum_Ozellikler | k=5 | Pencere=3 

,model_tipi,hisse_senedi,alt_kume,k_ozellik_sayisi,pencere_boyutu,dogruluk,model_dosyasi
1339,XGBoost,A,Tum_Ozellikler,5,7,0.525955,../models/xgb_A_subset-Tum_Ozellikler_k5_w7.jo...
1341,XGBoost,A,Tum_Ozellikler,10,7,0.503428,../models/xgb_A_subset-Tum_Ozellikler_k10_w7.j...
1343,XGBoost,A,Sadece_Teknik_Indikatorler,5,7,0.499510,../models/xgb_A_subset-Sadece_Teknik_Indikator...
1340,XGBoost,A,Tum_Ozellikler,10,3,0.493151,../models/xgb_A_subset-Tum_Ozellikler_k10_w3.j...
1338,XGBoost,A,Tum_Ozellikler,5,3,0.483366,../models/xgb_A_subset-Tum_Ozellikler_k5_w3.jo...
...,...,...,...,...,...,...,...
1348,XGBoost,ZTS,Sadece_Teknik_Indikatorler,5,3,0.497207,../models/xgb_ZTS_subset-Sadece_Teknik_Indikat...
1347,XGBoost,ZTS,Tum_Ozellikler,10,7,0.478992,../models/xgb_ZTS_subset-Tum_Ozellikler_k10_w7...
1345,XGBoost,ZTS,Tum_Ozellikler,5,7,0.467787,../models/xgb_ZTS_subset-Tum_Ozellikler_k5_w7....
1346,XGBoost,ZTS,Tum_Ozellikler,10,3,0.455307,../models/xgb_ZTS_subset-Tum_Ozellikler_k10_w3...
